In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# simulates a coin flip with head probability p 
def coin_flip(p):
    return np.random.choice(2, p=[1 - p, p])

In [2]:
def sim_pos(performance):
    # Simulates one possession in a game
    # Probability of en event in performance
    prob = [performance['ft_rate'],
         performance['fg2_rate'],
         performance['fg3_rate'],
         performance['tov_rate']]
    # One of the above events occur with corresponding probability
    event = np.random.choice(['ft', 'fg2', 'fg3', 'tov'], p=prob)
    # Return the result of the event playing out
    return event_outcome(event, performance)
    
def freethrow(performance):
    # Returns outcome of two free throws
    return coin_flip(performance['ft_pct']) + coin_flip(performance['ft_pct'])

def two(performance):
    # Returns outcome of a two point shot
    return 2 * coin_flip(performance['fg2_pct'])

def three(performance):
    # Returns outcome of a three point shot
    return 3 * coin_flip(performance['fg3_pct'])
    
def event_outcome(event, performance):
    if event == 'ft':
        return event, freethrow(performance)
    if event == 'fg2':
        return event, two(performance)
    if event == 'fg3':
        return event, three(performance)
    else:
        # There is a turnover, hence no points
        return event, 0

In [22]:
def sim_game(performance):
    # Simulates a games number of game
    # Average 100 possessions per team in NBA game
    outcomes = []
    for i in np.arange(100):
        event, points = sim_pos(performance)
        outcomes.append({'event': event, 'points': points})
    # turn outcomes into a DataFrame:
    game = pd.DataFrame(outcomes)
    return game

In [23]:
# Say your team is full of Steph Curries where you shoot
# 30% 2 pointers, 60% three pointers, turn the ball over 
# 5% of the time, and shoot free throws 5% of the time.

# A player has a 50% field goal percentage, 45% three point percentage
# and a 92% free throw percentage

# Then your performance is:
curries = pd.Series(
            [0.4, 0.5, 0.05, 0.05, .5, .45, .92],
        index=['fg2_rate', 'fg3_rate', 'ft_rate', 'tov_rate', 'fg2_pct', 'fg3_pct', 'ft_pct'])

curries

fg2_rate    0.40
fg3_rate    0.50
ft_rate     0.05
tov_rate    0.05
fg2_pct     0.50
fg3_pct     0.45
ft_pct      0.92
dtype: float64

In [26]:
# Simulating one game with a team full of Steph Curries
curry_game = sim_game(curries)

# Make sense of the results, maybe change the percentages of curries
# above and see what's happening to the results. 
# Notice the free throws are pretty much always going in (if there
# even are any since they only occur 5% of the time)!
curry_game.head(20)

,event,points
0,fg3,3
1,fg2,2
2,fg2,2
3,fg3,0
4,fg3,0
5,fg3,0
6,fg2,2
7,fg2,2
8,fg3,0
9,fg3,0


In [27]:
def game_results(performance):
    # Returns total points in a game
    results_table = sim_game(performance)
    
    # Results by play outcomes:
    ft = results_table[results_table['event'] == 'ft']
    fgs = results_table[results_table['event'].str.contains('fg')]
    fg3 = results_table[results_table['event'] == 'fg3']
    
    # Number in each play outcomes:
    ft_made = len(ft[ft['points'] != 0])
    fta = len(ft)
    fg_made = len(fgs[fgs['points'] != 0])
    fg3_made = len(fg3[fg3['points'] != 0])
    fga = len(fgs)
    total_points = sum(results_table['points'])
    
    return total_points

In [64]:
def win(points):
    if points > 100:
        return 1
    else:
        return 0

In [65]:
# Lets look at Curries team's wins:
wins = 0

for _ in range(82):
    curry_results = game_results(curries)
    if win(curry_results):
        print('Game won')
    else:
        print('Game lost')
    wins = wins + win(curry_results)
    print(wins)

Game won
1
Game won
2
Game won
3
Game won
4
Game won
5
Game lost
5
Game won
6
Game won
7
Game won
8
Game won
9
Game won
10
Game won
11
Game won
12
Game won
13
Game won
14
Game won
15
Game won
16
Game won
17
Game won
18
Game won
19
Game won
20
Game won
21
Game won
22
Game won
23
Game won
24
Game won
25
Game won
26
Game won
27
Game won
28
Game won
29
Game lost
29
Game won
30
Game won
31
Game lost
31
Game won
32
Game lost
32
Game won
33
Game won
34
Game won
35
Game won
36
Game won
37
Game won
38
Game won
39
Game won
40
Game won
41
Game won
42
Game won
43
Game won
44
Game won
45
Game won
46
Game won
47
Game lost
47
Game won
48
Game won
49
Game won
50
Game won
51
Game won
52
Game won
53
Game won
54
Game won
55
Game won
56
Game won
57
Game lost
57
Game won
58
Game won
59
Game lost
59
Game won
60
Game won
61
Game won
62
Game won
63
Game won
64
Game won
65
Game won
66
Game won
67
Game won
68
Game won
69
Game lost
69
Game won
70
Game won
71
Game won
72
Game won
73
Game won
74


In [66]:
def simulate_season(performance):
    # returns number of wins at end of season
    wins = 0
    
    for _ in range(82):
        result = game_results(performance)
        won = win(result)
        wins = wins + won
        
    print('Number of games won: ' + str(wins))

In [67]:
simulate_season(curries)

Number of games won: 75


In [68]:
def perf(fg2_rate, fg3_rate, ft_rate, tov_rate, fg2_pct, fg3_pct, ft_pct):
    return pd.Series({
            'fg2_rate': fg2_rate, 
            'fg3_rate': fg3_rate,
            'ft_rate': ft_rate,
            'tov_rate': tov_rate,
            'fg2_pct': fg2_pct, 
            'fg3_pct': fg3_pct,
            'ft_pct': ft_pct
            })

In [69]:
def rate_calculator(fta, fg2a, fg3a, orb, opp_drb, tov):
    possessions = fg2a + fg3a + tov + 0.4 * fta
    ft_rate = (0.4 * fta)/possessions
    fg2_rate = fg2a/possessions
    fg3_rate = fg3a/possessions
    oreb_rate = orb/(orb + opp_drb)
    tov_rate = tov/possessions
    
    print('ft rate ' + str(ft_rate))
    print('fg2 rate ' + str(fg2_rate))
    print('fg3 rate ' + str(fg3_rate))
    print('oreb rate ' + str(oreb_rate))
    print('tov rate ' + str(tov_rate))

In [70]:
# Warriors 
rate_calculator(20.4, 55.3, 34.4, 9.7, 33.5, 14.3)

ft rate 0.07275320970042798
fg2 rate 0.4930456490727533
fg3 rate 0.30670470756062773
oreb rate 0.224537037037037
tov rate 0.12749643366619118


In [72]:
warriors = perf(0.493, 0.307, 0.073, 0.127, 0.557, 0.385, .801)
simulate_season(warriors)

Number of games won: 47


In [75]:
# Lakers 
rate_calculator(16.3, 59.6, 31, 10.2, 35.6, 15.7)

ft rate 0.05779117177805355
fg2 rate 0.5282751285233115
fg3 rate 0.2747739762453466
oreb rate 0.22270742358078602
tov rate 0.13915972345328842


In [90]:
lakers = perf(0.527, 0.275, 0.058, 0.14, 0.541, 0.333, .699)
simulate_season(lakers)

Number of games won: 23


In [91]:
# Clippers
rate_calculator(22.6, 61.7, 25.8, 9.7, 34.5, 14.5)

ft rate 0.0814121037463977
fg2 rate 0.5556556195965417
fg3 rate 0.23234870317002881
oreb rate 0.2194570135746606
tov rate 0.1305835734870317


In [97]:
clips = perf(0.556, 0.232, 0.081, 0.131, 0.507, 0.388, .792)
simulate_season(clips)

Number of games won: 25
